In [1]:
import rpy2.rinterface_lib.callbacks
import logging
import rpy2.robjects as ro

from rpy2.robjects import pandas2ri
import anndata2ri
from rpy2.robjects.conversion import localconverter

pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [2]:
%%R
library(anndata)
library(Matrix)
library(CellChat)


    consider that it could be called from a Python process. This
    results in a quasi-obligatory segfault when rpy2 is evaluating
    R code using it. On the hand, rpy2 is accounting for the
    fact that it might already be running embedded in a Python
    process. This is why:
    - Python -> rpy2 -> R -> reticulate: crashes
    - R -> reticulate -> Python -> rpy2: works

    The issue with reticulate is tracked here:
    https://github.com/rstudio/reticulate/issues/208
    

R[write to console]: Loading required package: dplyr

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: igraph

R[write to console]: 
Attaching package: ‘igraph’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


R[write to console]: The following objects are masked from ‘package:stats’:

    decompose, spectrum


R[write to console]: The following object is masked from ‘package:base’:

    union


R[write to console]: Loading required package: ggplot2



In [3]:
%%R 
cellchat.early <- readRDS('objects/cellchat_early.rds')
cellchat.mid <- readRDS('objects/cellchat_mid.rds')
cellchat.late <- readRDS('objects/cellchat_late.rds')

In [4]:
%%R 
cellchat <- readRDS('objects/cellchat_merged.rds')

cellchat

An object of class CellChat created from a merged object with multiple datasets 
 969 signaling genes.
 114234 cells. 
CellChat analysis of single cell RNA-seq data! 


In [5]:
%%R

names(cellchat@net$early)

[1] "prob"   "pval"   "count"  "weight"


In [6]:
%%R

names(cellchat@net) <- c('GW 10-13', 'GW 14-16', 'GW 17-19')
names(cellchat@net)

[1] "GW 10-13" "GW 14-16" "GW 17-19"


In [7]:
%%R

cellchat@meta$datasets = factor(cellchat@meta$datasets, levels = c("Early", "Mid", 'Late')) # set factor level

In [8]:
%%R 
names(cellchat@netP) <- c('GW 10-13', 'GW 14-16', 'GW 17-19')

In [8]:
%%R 
names(cellchat@net)
gg1 <- rankNet(cellchat, mode = "comparison", stacked = TRUE, do.stat = FALSE, 
               measure='weight',show.raw=TRUE,do.flip=F,
               comparison = c(1, 2,3), 
               sources.use=c('Airway SMC', 'Lipofibroblast precursors',
                       'SOX2highCFTR+ cells',
                       'Basal cells', 'Club cells',
                        'Airway fibroblast progenitors'),
              targets.use=c('SCGB3A2hiSFTPBhiCFTRhi cells'))

pdf("./figures/interaction_strength_comparison_TP_Target_selected_sources.pdf", 
    width = 7, height = 2.5)
gg1 + theme(text = element_text(colour = "black"),
           axis.text.x = element_text(colour = "black"),
            axis.text.y = element_text(colour = "black"))

R[write to console]: The text on the y-axis will not be colored for the number of compared datasets larger than 3!

R[write to console]: The text on the y-axis will not be colored for the number of compared datasets larger than 3!



In [11]:
%%R
df = rankNet(cellchat, mode = "comparison", stacked = TRUE, do.stat = FALSE, 
               measure='weight',show.raw=TRUE,do.flip=F,
               comparison = c(1, 2,3), 
               sources.use=c('Airway SMC', 'Lipofibroblast precursors',
                       'SOX2highCFTR+ cells',
                       'Basal cells', 'Club cells',
                        'Airway fibroblast progenitors'),
               #signaling=c('FGF'),
              targets.use=c('SCGB3A2hiSFTPBhiCFTRhi cells'),return.data=T)$signaling.contribution

write.csv(df, 'data/cellchat_all_pathways.csv')

R[write to console]: The text on the y-axis will not be colored for the number of compared datasets larger than 3!

